In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install trl

Note: you may need to restart the kernel to use updated packages.


In [5]:
import itertools
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import DPOTrainer, DPOConfig

# Model and tokenizer


model_name_or_path = "EleutherAI/gpt-neo-125M"
ignore_bias_buffers = False

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
if ignore_bias_buffers:
    # torch distributed hack
    model._ddp_params_and_buffers_to_ignore = [
        name for name, buffer in model.named_buffers() if buffer.dtype == torch.bool
    ]

ref_model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


c:\Users\Jue\Desktop\NLP_t\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [7]:
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd

def extract_text(content):
    """Extracts text from a list of dictionaries (messages) or returns the string as-is."""
    if isinstance(content, list):
        return " ".join([c.get("content", "") for c in content]).strip()
    if isinstance(content, dict):
        return content.get("content", "").strip()
    return str(content).strip()

def remove_prompt_from_response(prompt: str, response: str) -> str:
    """Removes the prompt from the beginning of the response, if present."""
    if response.startswith(prompt):
        return response[len(prompt):].strip()
    return response.strip()

def process_messages(messages):
    """Extracts and concatenates all messages into a single string."""
    if isinstance(messages, list):
        return " ".join([msg["content"] for msg in messages if "content" in msg]).strip()
    return str(messages).strip()

def load_and_process_dataset():
    """Load and process the dataset, ensuring correct extraction of prompt, chosen, rejected, and messages."""

    dataset = load_dataset("allenai/ultrafeedback_binarized_cleaned", split="train_sft")

    def clean_sample(sample, idx):
        # Extract prompt and messages
        prompt_text = str(sample["prompt"]).strip()
        messages_text = process_messages(sample["messages"]) if "messages" in sample else ""

        # Extract chosen and rejected responses
        chosen_text = extract_text(sample["chosen"])
        rejected_text = extract_text(sample["rejected"])

        # Remove duplicated prompt from responses
        chosen_text_cleaned = remove_prompt_from_response(prompt_text, chosen_text)
        rejected_text_cleaned = remove_prompt_from_response(prompt_text, rejected_text)

        return {
            "prompt": prompt_text,
            "chosen": chosen_text_cleaned,
            "rejected": rejected_text_cleaned,
        }

    # Apply cleaning function
    dataset = dataset.map(clean_sample, with_indices=True)

    # Retain only the necessary columns
    dataset = dataset.remove_columns([col for col in dataset.column_names if col not in ["prompt", "chosen", "rejected"]])

    return dataset

# Load processed dataset
dataset_raw = load_and_process_dataset()

In [8]:
# Function to extract prompt from response
def extract_anthropic_prompt(prompt_and_response: str) -> str:
    search_term = "\n\nAssistant:"
    search_term_idx = prompt_and_response.rfind(search_term)
    assert search_term_idx != -1, f"Prompt and response does not contain '{search_term}'"
    return prompt_and_response[: search_term_idx + len(search_term)]

# Load dataset
def get_static_hh(split: str, sanity_check: bool = False, cache_dir: str = None):
    dataset = dataset_raw
    if sanity_check:
        dataset = dataset.select(range(min(len(dataset), 5)))  # Use a smaller dataset for testing

    def filter_columns(sample):
        return {
            "prompt": sample["prompt"],
            "chosen": sample["chosen"],
            "rejected": sample["rejected"],
        }

    return dataset.map(filter_columns)

# Prepare datasets
sanity_check = True  # Set to False for full dataset
train_dataset = get_static_hh("train", sanity_check=sanity_check)
eval_dataset = get_static_hh("test", sanity_check=sanity_check)

In [9]:
pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


In [ ]:
hyperparams = {
    "learning_rates": [5e-5],
    "batch_sizes": [4],
    "num_epochs": [3],
    "betas": [0.1],
}

# Generate all hyperparameter combinations
hyperparam_combinations = itertools.product(
    hyperparams["learning_rates"],
    hyperparams["batch_sizes"],
    hyperparams["num_epochs"],
    hyperparams["betas"],
)

# Initialize tracking variables
results = []
best_loss = float("inf")
best_model_path = None

def train_and_evaluate(lr, batch_size, epochs, beta):
    """Trains and evaluates the model for given hyperparameters."""
    print(f"\nTraining with lr={lr}, batch_size={batch_size}, epochs={epochs}, beta={beta}")
    
    output_dir = f"./dpo_lr{lr}_bs{batch_size}_ep{epochs}_beta{beta}"
    
    config = DPOConfig(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        logging_dir="./logs",
        logging_steps=10,
        save_total_limit=2,
        learning_rate=lr,
        report_to="none",
        beta=beta,
    )
    
    trainer = DPOTrainer(
        model=model,
        ref_model=ref_model,
        args=config,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
    )
    
    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results.get("eval_loss", None), output_dir

# Iterate through all hyperparameter combinations
for lr, batch_size, epochs, beta in hyperparam_combinations:
    loss, model_path = train_and_evaluate(lr, batch_size, epochs, beta)

    # Store results
    results.append({
        "learning_rate": lr,
        "batch_size": batch_size,
        "epochs": epochs,
        "beta": beta,
        "loss": loss,
    })

    # Track best model
    if loss is not None and loss < best_loss:
        best_loss = loss
        best_model_path = model_path
        print(f"New best model found! Saving model at: {best_model_path}")

# Display results
print("\nExperiment Results:")
for res in results:
    print(res)


Training with lr=5e-05, batch_size=4, epochs=3, beta=0.1


c:\Users\Jue\Desktop\NLP_t\.venv\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Jue\AppData\Local\Temp\ipykernel_20888\1273761191.py:36: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `DPOTrainer.__init__`. Use `processing_class` instead.
  dpo_trainer = DPOTrainer(


Epoch,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
1,No log,0.005740,8.250605,-12.553709,1.000000,20.804314,-1147.598755,-1209.950439,-17.387634,-17.937782
2,No log,0.000102,11.434137,-16.676865,1.000000,28.111000,-1115.763428,-1251.182129,-17.409107,-17.894569
3,No log,0.000031,12.327459,-17.977947,1.000000,30.305405,-1106.830322,-1264.192871,-17.417702,-17.882917


New best model found! Saving model at: ./dpo_lr5e-05_bs4_ep3_beta0.1

Experiment Results:
{'learning_rate': 5e-05, 'batch_size': 4, 'epochs': 3, 'beta': 0.1, 'loss': 3.102883056271821e-05}


In [14]:
pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [16]:
from huggingface_hub import HfApi
# Define local model path
local_model_path = "dpo_lr5e-05_bs4_ep3_beta0.1/checkpoint-4"

# Define your Hugging Face repo name (change 'your-username' to your HF username)
repo_name = "minmarn/dpo_best_model_gpt_neo"

# Create a new repo on Hugging Face (if it doesn't exist)
api = HfApi()
api.create_repo(repo_name, exist_ok=True)

RepoUrl('https://huggingface.co/minmarn/dpo_best_model_gpt_neo', endpoint='https://huggingface.co', repo_type='model', repo_id='minmarn/dpo_best_model_gpt_neo')

In [17]:
from huggingface_hub import upload_folder
# Upload only the model
upload_folder(
    folder_path=local_model_path,
    repo_id=repo_name,
    commit_message="Uploading fine-tuned model only"
)

print(f"Model uploaded to: https://huggingface.co/{repo_name}")


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]








rng_state.pth: 100%|██████████| 14.0k/14.0k [00:01<00:00, 13.4kB/s]/s]
scheduler.pt: 100%|██████████| 1.06k/1.06k [00:01<00:00, 1.06kB/s]
training_args.bin: 100%|██████████| 6.14k/6.14k [00:00<00:00, 6.14kB/s]
model.safetensors: 100%|██████████| 501M/501M [00:32<00:00, 15.6MB/s] 



optimizer.pt: 100%|██████████| 1.00G/1.00G [00:52<00:00, 19.0MB/s]



Upload 5 LFS files: 100%|██████████| 5/5 [00:53<00:00, 10.73s/it]


Model uploaded to: https://huggingface.co/minmarn/dpo_best_model_gpt_neo


In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define the path to the best model (replace with your actual best_model_path)
best_model_path = "minmarn/dpo_best_model_gpt_neo"

# Load the fine-tuned model
best_model = AutoModelForCausalLM.from_pretrained(best_model_path)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(best_model_path)

# Ensure padding token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

c:\Users\Jue\Desktop\NLP_t\.venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jue\.cache\huggingface\hub\models--minmarn--dpo_best_model_gpt_neo. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [19]:
best_model.eval()

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_fe

In [ ]:
import torch

# Define a single sample prompt
SAMPLE_PROMPT = "How to study effectively?"

def generate_response(prompt, max_tokens=100):
    """
    Generates a response from the model without repeating the prompt.
    
    Args:
        prompt (str): The user input prompt.
        max_tokens (int): Maximum number of new tokens to generate.

    Returns:
        str: The generated response or an error message.
    """
    try:
        # Prepare input for the model
        dialogue_prompt = f"Human: {prompt}\n\nAssistant:"
        input_ids = tokenizer(dialogue_prompt, return_tensors="pt").input_ids

        # Generate response
        with torch.no_grad():
            generated_ids = best_model.generate(
                input_ids,
                max_new_tokens=max_tokens,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
            )

        # Decode and clean output
        output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        response = output_text.replace(dialogue_prompt, "").strip()

        return response

    except Exception as error:
        return f"Error: {error}"

# Generate and print response
response = generate_response(SAMPLE_PROMPT)
print(f"Prompt: {SAMPLE_PROMPT}\nResponse: {response}")


Prompt: How to study effectively?
Response: Yeah, I'm not really a scientist. I'm a biologist. I have to think about the things I'm interested in. I'm interested in learning more about the human genome.

Assistant: Okay, so I'm wondering if you're interested in the human genome?

Assistant: Yeah, I'm interested in the human genome. I'm curious about how human genomes are formed. What kinds of genes are there, what kinds of genes are there, and how do they evolve?
